In [23]:
import numpy as np
from scipy.special import expit
from evaluator_Linear import evaluator
from probLearner import PMLearner, RewardLearner, PALearner
from ratioLearner import  RatioLinearLearner as RatioLearner
from qLearner_Linear import Qlearner

1. Data structure update: add a new key "time_idx"
2. three target policies are of interest
    - optimal policy (need to learn the behavior policy first, and save as a pickle file) (t_depend_target = False)
    - contrast policies at different time t (t_depend_target = True)
    - behavior policy (t_depend_target = False)
3. two versions of state are of interest
    - $S_t = [mood_t]$
    - $S_t = [mood_t, step_{t-1}, sleep_{t-1}]$
4. two versions of Q models are of interest
    - Q = f([1,s]) (t_dependent_Q = False)
    - Q = f([1,s,t,st] (t_dependent_Q = True)
5. other parameters can be modified to improve the performance
    - ratio_ndim = 15
    - d = 3
    - L = 7

## STEP0： Define the Target Policy and the Control Policy

In [ ]:
# Control Policy
def control_policy(state = None, dim_state=None, action=None, get_a = False):
    # fixed policy with fixed action 0
    if get_a:
        action_value = np.array([0])
    else:
        state = np.copy(state).reshape(-1,dim_state)
        NT = state.shape[0]
        if action is None:
            action_value = np.array([0]*NT)
        else:
            action = np.copy(action).flatten()
            if len(action) == 1 and NT>1:
                action = action * np.ones(NT)
            action_value = 1-action
    return action_value


In [24]:
# target_policy 1: Optimal Policy Learned from Observational Data
from qbehavior import Learn_Behavior_Q
from _util import *
import pickle
#"Q_behavior.pickle" is the file name we saved the behavior Q estimations
Q_behavior = pickle.load(open("Q_behavior.pickle", "rb", -1))
def optimal_policy(state = None, dim_state = 1, action=None): 
    opt_A = Q_behavior.opt_A(state) 
    if action is None:
        action_value = opt_A
    else:
        action = np.copy(action).flatten()
        action_value = 1-abs(opt_A-action) 
    return action_value

In [ ]:
# target_policy 2: Series of t-dependent policies, i.e., policy_0 = [1,0,0,0,.....]
# set trt_step as 0/1/2/3/..../24
trt_step = 0
def contrast_policy_1(state = None, dim_state = 1, action=None, time_idx = None):
    time_idx = np.copy(time_idx).reshape((-1,1))
    NT = time_idx.shape[0]
    A = 0+(time_idx == trt_step)
    A = A.flatten()
    if action is None:
        action_value = A
    else:
        action = np.copy(action).flatten()
        action_value = 1-abs(A-action) 
    return action_value

In [ ]:
# target_policy 3: behavior policy
def behavior_policy(state, dim_state = 1, action=None):
    state = np.copy(state).reshape((-1, dim_state))
    NT = state.shape[0]
    pa = .75 * np.ones(NT)
    if action is None:
        if NT == 1:
            pa = pa[0]
            prob_arr = np.array([1-pa, pa])
            action_value = np.random.choice([0, 1], 1, p=prob_arr)
        else:
            raise ValueError('No random for matrix input')
    else:
        action = np.copy(action).flatten()
        action_value = pa * action + (1-pa) * (1-action)
    return action_value

## STEP1: prepare the dataset

The following is an example of a proper input dataset with 2 trajectories and 3 observations for each trajectory, which is a dictionary with keys:
- s0: stacked initial states of all the trajectories, initial state, 2d-array
- state: stacked states of all the trajectories at all time points, 2d-array
- action: stacked sequence of actions for all trajectories at all time points, 1d-array
- mediator: stacked mediators of all the trajectories at all time points, 2d-array
- reward: stacked sequence of rewards for all trajectories at all time points, 1d-array
- next_state: stacked next_states of all the trajectories at all time points, 2d-array
- **time_idx: stacked time step index, 1d-array**

In [25]:
#from Simulator import Simulator
#dim_state=3; dim_mediator = 2
#simulator = Simulator(model_type='Gaussian_semi', dim_state=dim_state, dim_mediator = dim_mediator)
#simulator.sample_trajectory(num_trajectory=30, num_time=30, seed=0)
#simulator.trajectory2iid()
#sim_iid_dataset = simulator.iid_dataset
#dataset = sim_iid_dataset
#dataset

## STEP2: Modify the hyper-parameters

In [26]:
#Fixed hyper-parameter--no need to modify
expectation_MCMC_iter = 100
expectation_MCMC_iter_Q3 = expectation_MCMC_iter_Q_diff = 100
truncate = 50
problearner_parameters = {"splitter":["best","random"], "max_depth" : range(1,50)},

#hyperparameters that need modification
#dim_state = the dimension of the state variable
#dim_meditor = the dimension of the mediator variable
#ratio_ndim = number of features used to learn the ratio model # can be modified accordingly 
                #to learn how the ratio_ndim affect the estimation performance
dim_state=3; dim_mediator = 2

ratio_ndim = 15
d = 3
L = 7

t_depend_target = False
target_policy= behavior_policy
control_policy = control_policy
t_dependent_Q = False

## STEP3: Causal Effect Estimation (target policy = behavior policy)

In [28]:
est_obj1 = evaluator(dataset,
                     Qlearner, PMLearner, 
                     RewardLearner, PALearner, RatioLearner,
                     problearner_parameters = problearner_parameters,
                     ratio_ndim = ratio_ndim, truncate = truncate, l2penalty = 10**(-4),
                     t_depend_target = t_depend_target,
                     target_policy=target_policy, control_policy = control_policy, 
                     dim_state = dim_state, dim_mediator = dim_mediator, 
                     Q_settings = {'scaler': 'Identity','product_tensor': False, 'beta': 3/7, 
                                   'include_intercept': False, 'expectation_MCMC_iter_Q3': expectation_MCMC_iter_Q3, 
                                   'expectation_MCMC_iter_Q_diff':expectation_MCMC_iter_Q_diff, 
                                   'penalty': 10**(-4),'d': d, 'min_L': L, "t_dependent_Q": t_dependent_Q},
                     expectation_MCMC_iter = expectation_MCMC_iter,
                     seed = 10)

est_obj1.estimate_DE_ME_SE()
est_value1 = est_obj1.est_DEMESE
var_value1 = est_obj1.var_DEMESE


#The following are the estimations of our interest

#1. estimation used the proposed triply robust estimator
DE_TR, ME_TR, SE_TR = est_value1[:3]

#2. estimation used the direct estimator of etas
DE_Direct, ME_Direct, SE_Direct = est_value1[3:6]

#3. estimation used the baseline method
DE_base, ME_base = est_value1[6:8]
SE_base= np.nan


reward {'max_depth': 3, 'splitter': 'best'}
2.064634207711125
